## Background

During large-scale date training, the order of magnitude of data can reach millions. If a parameter is acquired via the computation of the whole training set, the update speed will be too slow. To solve this problem, a common used method is [Mini-Batch Gradient Descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) which computes mini-batche data in the training set, resulting faster training of parameters in a neural network.

In this article, we will first define a softmax classifier, then use the training set of [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) to train this neural network, and finally use the test set to verify the accuracy of the neural network. The difference is that we will use Mini-Batch Gradient Descent, thus the accuracy of the neural network can reach 40%.

## This article is the same as the last article. Import dependencies and build the neural network.

In [1]:
import $plugin.$ivy.`com.thoughtworks.implicit-dependent-type::implicit-dependent-type:2.0.0`

import $ivy.`com.thoughtworks.deeplearning::differentiableany:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiablenothing:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiableseq:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiabledouble:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiablefloat:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiablehlist:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiablecoproduct:1.0.0`
import $ivy.`com.thoughtworks.deeplearning::differentiableindarray:1.0.0`
import $ivy.`org.nd4j:nd4j-native-platform:0.7.2`
import $ivy.`org.rauschig:jarchivelib:0.5.0`

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import java.io.{FileInputStream, InputStream}


import com.thoughtworks.deeplearning
import org.nd4j.linalg.api.ndarray.INDArray
import com.thoughtworks.deeplearning.DifferentiableHList._
import com.thoughtworks.deeplearning.DifferentiableDouble._
import com.thoughtworks.deeplearning.DifferentiableINDArray._
import com.thoughtworks.deeplearning.DifferentiableAny._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._
import com.thoughtworks.deeplearning.Layer.Tape
import com.thoughtworks.deeplearning.Symbolic.Layers.Identity
import com.thoughtworks.deeplearning.Symbolic._
import com.thoughtworks.deeplearning.{
  DifferentiableHList,
  DifferentiableINDArray,
  Layer,
  Symbolic
}
import com.thoughtworks.deeplearning.Poly.MathFunctions._
import com.thoughtworks.deeplearning.Poly.MathMethods./
import com.thoughtworks.deeplearning.Poly.MathOps
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.indexing.{INDArrayIndex, NDArrayIndex}
import org.nd4j.linalg.ops.transforms.Transforms
import org.nd4s.Implicits._
import shapeless._

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import scala.collection.immutable.IndexedSeq
import scala.util.Random

pprintConfig() = pprintConfig().copy(height = 2)

import $file.ReadCIFAR10ToNDArray
import $file.Utils

def softmax(implicit scores: INDArray @Symbolic): INDArray @Symbolic = {
  val expScores = exp(scores)
  expScores / expScores.sum(1)
}

implicit def optimizer: Optimizer = new LearningRate {
  def currentLearningRate() = 0.00001
}

//10 label of CIFAR10 images(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val NumberOfClasses: Int = 10
val NumberOfPixels: Int = 3072

def createMyNeuralNetwork(implicit input: INDArray @Symbolic): INDArray @Symbolic = {
  val initialValueOfWeight = Nd4j.randn(NumberOfPixels, NumberOfClasses) * 0.001
  val weight: INDArray @Symbolic = initialValueOfWeight.toWeight
  val result: INDArray @Symbolic = input dot weight
  softmax.compose(result)
}
val myNeuralNetwork = createMyNeuralNetwork

def lossFunction(implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val input = pair.head
  val expectedOutput = pair.tail.head
  val probabilities = myNeuralNetwork.compose(input)

  -(expectedOutput * log(probabilities)).mean
}

Compiling ReadCIFAR10ToNDArray.sc
Compiling Utils.sc


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


import $plugin.$                                                                             


import $ivy.$                                                       

import $ivy.$                                                           

import $ivy.$                                                       

import $ivy.$                                                          

import $ivy.$                                                         

import $ivy.$                                                         

import $ivy.$                                                             

import $ivy.$                                                            

import $ivy.$                                    

import $ivy.$                               


import $ivy.$                                             


import java.io.{FileInputStream, InputStream}



import com.thoughtworks.deeplearning

import org.nd4j.linalg.api.ndarray.INDArray

import com.thoughtworks.deeplea

## Apply Mini-Batch Gradient Descent

Like the previous article, we need to train the neural network. However, the difference is that the training data in this article are read randomly, and in the last article one batch of data set is used for its repeated training. Train the neural network and observe the change of `loss` in each training. The trend of `loss` is decrease, however, we need to find out whether it decreases every time. (The future is bright, the road is tortuous.)

### Read and process the data according to the array, and then train the neural network.

In [2]:
val MiniBatchSize = 256

def trainData(randomIndexArray: Array[Int]): Double = {
  val trainNDArray :: expectLabel :: shapeless.HNil =
    ReadCIFAR10ToNDArray.getSGDTrainNDArray(randomIndexArray)

  val input =
    trainNDArray.reshape(MiniBatchSize, NumberOfPixels)

  val expectLabelVectorized =
    Utils.makeVectorized(expectLabel, NumberOfClasses)

  lossFunction.train(input :: expectLabelVectorized :: HNil)
}

MiniBatchSize: Int = 256
defined function trainData

### Disrupt the order of a sequence once for each [epoch](http://stackoverflow.com/questions/4752626/epoch-vs-iteration-when-training-neural-networks), and generate the random arrays.

In [3]:
val random = new Random

val lossSeq =
  (
    for (iteration <- 0 to 50) yield {
      val randomIndex = random
        .shuffle[Int, IndexedSeq](0 until 10000) //https://issues.scala-lang.org/browse/SI-6948
        .toArray
      for (times <- 0 until 10000 / MiniBatchSize) yield {
        val randomIndexArray =
          randomIndex.slice(times * MiniBatchSize,
                            (times + 1) * MiniBatchSize)
          val loss = trainData(randomIndexArray)
          if(times == 3 & iteration % 5 == 4){
            println("at epoch " + (iteration / 5 + 1) + " loss is :" + loss)
          }
          loss
      }
    }
  ).flatten

plotly.JupyterScala.init()

val plot = Seq(
  Scatter(lossSeq.indices, lossSeq)
)

plot.plot(
  title = "loss by time"
)

at epoch 1 loss is :0.21291561126708985
at epoch 2 loss is :0.20125489234924315
at epoch 3 loss is :0.19702608585357667
at epoch 4 loss is :0.1977776288986206
at epoch 5 loss is :0.19166061878204346
at epoch 6 loss is :0.198227858543396
at epoch 7 loss is :0.1935807228088379
at epoch 8 loss is :0.20156009197235109
at epoch 9 loss is :0.18221101760864258
at epoch 10 loss is :0.18732290267944335


random: Random = scala.util.Random@320757a
lossSeq: IndexedSeq[Double] = Vector(
  0.23041152954101562,
...
plot: Seq[Scatter] = List(
  Scatter(
...
res2_4: String = "plot-314683916"

## Prepare and process the test set

Like [the previous article](https://thoughtworksinc.github.io/DeepLearning.scala/demo/SoftmaxLinearClassifier.html), we read the images and corresponding label information for test data from CIFAR10 database and process them. However, here we only read the test set, and the training set is randomly read during training.

In [4]:
val testNDArray =
   ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

val testData = testNDArray.head

val testExpectResult = testNDArray.tail.head

val vectorizedTestExpectResult = Utils.makeVectorized(testExpectResult, NumberOfClasses)

testNDArray: INDArray :: INDArray :: HNil = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....
testData: INDArray = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0....
testExpectResult: INDArray = [3.00, 8.00, 8.00, 0.00, 6.00, 6.00, 1.00, 6.00, 3.00, 1.00, 0.00, 9.00, 5.00, 7.00, 9.00, 8.00, 5.00, 7.00, 8.00, 6.00, 7.00, 0.00, 4.00, 9.00, 5.00, 2.00, 4.0...
vectorizedTestExpectResult: INDArray = [[0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
...

## Verify the neural network and predict the accuracy.

Just like the last article, we use the test data to verify the prediction result of the neural network and compute the accuracy. This time, the accuracy may increase to about 40%.

In [5]:
val right = Utils.getAccuracy(myNeuralNetwork.predict(testData), testExpectResult)
println(s"the result is $right %")

the result is 37.0 %


right: Double = 37.0

## Summary

We have learned the follows in this article:

* Mini-Batch Gradient Descent
* epoch

[Complete code](https://github.com/izhangzhihao/deeplearning-tutorial/blob/master/src/main/scala/com/thoughtworks/deeplearning/tutorial/MiniBatchGradientDescent.scala)